In [5]:
using Documenter;
using Crayons;
using Logging;
using JuMP, MathOptInterface;
using Gurobi;
const MOI = MathOptInterface;
const MOIU = MOI.Utilities;

# LP Knapsack Problem

$$
\begin{array}{rl}
    \max & \mathbf{c}^{\small\mathbf{\text{T}}}\mathbf{x} \\
    \text{s.t.} & \mathbf{w}^{\small\mathbf{\text{T}}}\mathbf{x} \le C\\
    ~ & \mathbf{x}_i \in \{0, 1\}
\end{array}
$$

In [6]:
# Model definition
# References:
# [1] https://jump.dev/MathOptInterface.jl/stable/tutorials/example/

model = MOIU.Model{Float64}()

n = 3;
c = [1.0, 2.0, 3.0];  #  6.0
w = [3.0, 5.0, 10.0]; # 18.0 (32.0 - 18.0 = 14.0) (expansion = 2.0 + 4.0 + 8.0 = 14.0)
C = 32.0;

x = MOI.add_variables(model, n);

# ---------
# Objective
# ---------
MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)

MOI.set(
   model,
   MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}(),
   MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(c, x), 0.0),
);

# -----------
# Constraints
# -----------
MOI.add_constraint(
   model,
   MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(w, x), 0.0),
   MOI.LessThan(C),
);

for xᵢ in x
   MOI.add_constraint(model, xᵢ, MOI.ZeroOne())
end

In [7]:
success() = println(crayon"#66ff66", "QUBO Success", crayon"#ffffff");
failure() = println(crayon"#ff6666", "QUBO Failure", crayon"#ffffff");

In [8]:
include("../src/ToQUBO.jl")

qubo = ToQUBO.toqubo(model)

ToQUBO.isqubo(qubo) ? success() : failure();

print(qubo.model);

gurobi = Gurobi.Optimizer();
varmap = MOI.copy_to(gurobi, qubo.model);
MOI.optimize!(gurobi);

for xᵢ in MOI.get(gurobi, MOI.ListOfVariableIndices())
    vᵢ = varmap[xᵢ]
    if ToQUBO.isslack(qubo.target[vᵢ])
        var = "s"
    else
        var = "x"
    end
    println(ToQUBO.subscript(vᵢ, var=var), " = ", MOI.get(gurobi, MOI.VariablePrimal(), xᵢ));
end

QUBO Success
Set parameter Username
Academic license - for non-commercial use only - expires 2022-02-02
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 0 rows, 8 columns and 0 nonzeros
Model fingerprint: 0x62ca5f1b
Model has 28 quadratic objective terms
Variable types: 0 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [8e-02, 1e+00]
  QObjective range [1e-02, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -1.3333333
Found heuristic solution: objective -0.3750000
Found heuristic solution: objective 0.0013021
Presolve time: 0.00s
Presolved: 28 rows, 36 columns, 84 nonzeros
Variable types: 0 continuous, 36 integer (36 binary)

Root relaxation: objective -3.997396e-01, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objectiv

┌ Info: Original Binary Variables: x₂ x₃ x₁
└ @ Main.ToQUBO c:\Users\pedroxavier\gits\ToQUBO.jl\src\ToQUBO.jl:222
┌ Info: Variables for expansion: ∅
└ @ Main.ToQUBO c:\Users\pedroxavier\gits\ToQUBO.jl\src\ToQUBO.jl:224
┌ Info: C = 32.0 (5 bits)
└ @ Main.ToQUBO c:\Users\pedroxavier\gits\ToQUBO.jl\src\ToQUBO.jl:361
